In [ ]:
# Cell 1: Chuẩn bị dữ liệu ban đầu và kiểm tra số lớp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import re # Thêm thư viện re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.preprocessing import LabelEncoder

from gensim.models import Word2Vec, FastText
import nltk

# Tải tài nguyên NLTK một cách an toàn
# Lưu ý: Trên Kaggle, nếu kernel không có internet, việc download sẽ thất bại.
# Hãy đảm bảo internet được bật cho notebook (Settings -> Internet: ON)
print("Đang kiểm tra và tải tài nguyên NLTK...")
try:
    nltk.data.find('corpora/wordnet.zip')
    print("WordNet đã có.")
except: # Bắt Exception chung vì nltk.downloader.DownloadError có thể không tồn tại
    print("Đang tải WordNet...")
    nltk.download('wordnet', quiet=True)
    print("Đã tải WordNet.")

try:
    nltk.data.find('tokenizers/punkt')
    print("Punkt tokenizer đã có.")
except:
    print("Đang tải Punkt tokenizer...")
    nltk.download('punkt', quiet=True)
    print("Đã tải Punkt tokenizer.")

try:
    nltk.data.find('corpora/stopwords')
    print("Stopwords đã có.")
except:
    print("Đang tải Stopwords...")
    nltk.download('stopwords', quiet=True)
    print("Đã tải Stopwords.")
print("Hoàn tất kiểm tra tài nguyên NLTK.")


from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 200)
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except OSError:
    print("Cảnh báo: Không tìm thấy style 'seaborn-v0_8-whitegrid'. Sử dụng style mặc định.")
    pass

# --- TẢI DỮ LIỆU THỰC ---
DATA_PATH = '/kaggle/input/legal-text-classification-dataset/legal_text_classification.csv' # Đường dẫn như trong file gốc
# Trên Kaggle, nếu bạn đã upload dataset và nó được mount vào /kaggle/input/your-dataset-name/
# thì đường dẫn có thể là: DATA_PATH = '/kaggle/input/your-dataset-name/legal_text_classification.csv'
# Hoặc nếu 'data' là một thư mục trong dataset của bạn:
# DATA_PATH = '/kaggle/input/your-dataset-name/data/legal_text_classification.csv'
# Hãy điều chỉnh DATA_PATH nếu cần thiết cho môi trường Kaggle của bạn.

try:
    df = pd.read_csv(DATA_PATH, on_bad_lines="skip", engine='python')
    print(f"Tải dữ liệu thành công từ {DATA_PATH}. Số dòng: {len(df)}")
except FileNotFoundError:
    print(f"LỖI: Không tìm thấy file dữ liệu tại '{DATA_PATH}'. Vui lòng kiểm tra lại đường dẫn.")
    print("Chương trình có thể không hoạt động đúng nếu không có dữ liệu.")
    # Tạo DataFrame rỗng để tránh lỗi ở các cell sau nếu muốn tiếp tục debug cấu trúc code
    df = pd.DataFrame(columns=['case_id', 'case_outcome', 'case_title', 'case_text'])
# --- KẾT THÚC TẢI DỮ LIỆU ---

# Xử lý giá trị thiếu (như trong notebook gốc)
df = df.fillna('')

# Tạo cột 'case_text_sum'
if 'case_title' in df.columns and 'case_text' in df.columns:
    df['case_text_sum'] = df['case_title'] + " " + df['case_text']
    print("Đã tạo cột 'case_text_sum'.")
else:
    print("Cảnh báo: Thiếu cột 'case_title' hoặc 'case_text'. Không thể tạo 'case_text_sum'.")
    if 'case_text_sum' not in df.columns: # Nếu chưa có, tạo cột rỗng để tránh lỗi sau này
        df['case_text_sum'] = ""


# Mã hóa nhãn
if 'case_outcome' in df.columns and not df['case_outcome'].empty:
    le = LabelEncoder()
    df['case_outcome_num'] = le.fit_transform(df['case_outcome'])
    num_classes = df['case_outcome_num'].nunique()
    print(f"Đã mã hóa 'case_outcome'. Số lượng lớp (num_classes): {num_classes}")
    print("Mapping nhãn -> số:")
    for i, class_name in enumerate(le.classes_):
        print(f"  {class_name} -> {i}")
else:
    print("Cảnh báo: Không tìm thấy cột 'case_outcome' hoặc cột rỗng. Đặt num_classes mặc định là 10.")
    num_classes = 10
    # Khởi tạo le rỗng để tránh lỗi nếu các cell sau cố gắng dùng le.classes_
    le = LabelEncoder()


# --- TIỀN XỬ LÝ VĂN BẢN ---
# Hàm làm sạch (tương tự notebook gốc, có chỉnh sửa một chút cho rõ ràng)
def clean_text_revised(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    # Loại bỏ URL trước, rồi mới loại bỏ ký tự đặc biệt
    text = re.sub(r'https?://\S+|www\.\S+', '', text) # Regex chính xác hơn cho URL
    
    # Loại bỏ các ký tự không phải chữ cái, số và khoảng trắng (giữ lại chữ và số)
    # text = re.sub(r'[^a-z0-9\s]', '', text) # Cách này có thể giữ lại số nếu muốn
    # Theo notebook gốc, loại bỏ cả số và nhiều ký tự đặc biệt:
    marks_and_digits = r'''!()-[]{};?@#$%:'"\\,|./^&;*_0123456789'''
    text = ''.join(char for char in text if char not in marks_and_digits)

    # Loại bỏ các cụm từ không mong muốn
    unwanted_phrases = ['url', 'privacy policy', 'disclaimers', 'disclaimer', 'copyright policy']
    for phrase in unwanted_phrases:
        text = text.replace(phrase, '')
        
    text = re.sub(r'\s+', ' ', text).strip() # Loại bỏ khoảng trắng thừa
    return text

print("Bắt đầu làm sạch cột 'case_text_sum'...")
df['clean_text'] = df['case_text_sum'].apply(clean_text_revised)
print("Hoàn thành làm sạch văn bản.")


# Hàm tokenize và áp dụng stemming/lemmatization
stop_words_list = nltk_stopwords.words('english')
porter_stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

def tokenize_and_process(text, stop_words, processor_func=None, processor_type=None):
    if not isinstance(text, str):
        return []
    tokens = text.split() # Tokenizer đơn giản bằng split
    # tokens = nltk.word_tokenize(text) # Lựa chọn tokenizer tốt hơn
    
    processed_tokens = []
    for word in tokens:
        if word not in stop_words and len(word) > 2: # Lọc stop words và từ ngắn
            if processor_type == 'stem' and processor_func:
                processed_tokens.append(processor_func(word))
            elif processor_type == 'lem' and processor_func:
                processed_tokens.append(processor_func(word, pos='v')) # Lemmatize động từ
            elif not processor_type: # Chỉ tokenize và loại bỏ stop words
                 processed_tokens.append(word)
            # else: processed_tokens.append(word) # Trường hợp không stem/lem nhưng vẫn giữ từ
    return processed_tokens

print("Bắt đầu tokenizing, stemming và lemmatization...")
df['tokens_stm'] = df['clean_text'].apply(lambda x: tokenize_and_process(x, stop_words_list, porter_stemmer.stem, 'stem'))
df['tokens_lem'] = df['clean_text'].apply(lambda x: tokenize_and_process(x, stop_words_list, wordnet_lemmatizer.lemmatize, 'lem'))

# Tạo lại chuỗi văn bản đã xử lý
df['text_stm_joined'] = df['tokens_stm'].apply(' '.join)
df['text_lem_joined'] = df['tokens_lem'].apply(' '.join)
print("Hoàn thành tokenizing, stemming, lemmatization và tạo chuỗi văn bản.")
print("\nDataFrame head sau tiền xử lý cơ bản:")
print(df[['case_outcome', 'case_outcome_num', 'clean_text', 'tokens_stm', 'tokens_lem']].head(2))
# --- KẾT THÚC PHẦN TIỀN XỬ LÝ ---

In [ ]:
# dataset_path = '/kaggle/input/legal-text/cleaned_legal_text.csv'
# df = pd.read_csv(dataset_path)

In [ ]:
# le = LabelEncoder()
# df['case_outcome_num'] = le.fit_transform(df['case_outcome'])

In [ ]:
# Dữ liệu cho stemming
X_stm_tokens = df['tokens_stm'] # Series of lists of tokens
X_stm_text = df['text_stm_joined'] # Series of strings
y_stm = df['case_outcome_num']

In [ ]:
# Dữ liệu cho lemmatization
X_lem_tokens = df['tokens_lem'] # Series of lists of tokens
X_lem_text = df['text_lem_joined'] # Series of strings
y_lem = df['case_outcome_num']

In [ ]:
# Train-test split cho stemming data
train_stm_tokens, test_stm_tokens, train_stm_text, test_stm_text, y_train_stm, y_test_stm = train_test_split(
    X_stm_tokens, X_stm_text, y_stm, test_size=0.2, random_state=42, stratify=y_stm
)

In [ ]:
# Train-test split cho lemmatization data
train_lem_tokens, test_lem_tokens, train_lem_text, test_lem_text, y_train_lem, y_test_lem = train_test_split(
    X_lem_tokens, X_lem_text, y_lem, test_size=0.2, random_state=42, stratify=y_lem
)

In [ ]:
print("Kích thước tập huấn luyện (stemming tokens):", len(train_stm_tokens))
print("Kích thước tập kiểm tra (stemming tokens):", len(test_stm_tokens))
print("Kích thước tập huấn luyện (lemmatization text):", len(train_lem_text))
print("Kích thước tập kiểm tra (lemmatization text):", len(test_lem_text))

In [ ]:
import joblib # Để lưu model scikit-learn
import os

# Tạo thư mục để lưu models nếu chưa có
MODEL_SAVE_DIR = "saved_models"
if not os.path.exists(MODEL_SAVE_DIR):
    os.makedirs(MODEL_SAVE_DIR)

best_f1_score_global = -1.0 # Theo dõi F1 score tốt nhất qua tất cả các mô hình
best_model_path_global = ""
best_model_type_global = "" # 'sklearn' hoặc 'pytorch'

def compute_metrics_updated(y_true, y_pred, model_name=""):
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='macro')
    print(f"Kết quả cho {model_name}:")
    print(f"  Accuracy: {accuracy*100:.3f}%")
    print(f"  F1 Score (Macro): {f1*100:.3f}%")
    
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(max(6, num_classes // 2), max(4, num_classes // 2.5))) # Điều chỉnh kích thước dựa trên số lớp
    sns.heatmap(cm, annot=True, fmt='d', cmap='YlGnBu', 
                xticklabels=le.classes_ if 'le' in globals() and hasattr(le, 'classes_') else range(num_classes), 
                yticklabels=le.classes_ if 'le' in globals() and hasattr(le, 'classes_') else range(num_classes))
    plt.title(f'Confusion Matrix - {model_name}')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.tight_layout()
    plt.show()
    return accuracy, f1

def train_and_evaluate_ml_model(model, X_train, y_train, X_test, y_test, model_name=""):
    global best_f1_score_global, best_model_path_global, best_model_type_global # Khai báo sử dụng biến global

    print(f"\n--- Huấn luyện và Đánh giá: {model_name} ---")
    model.fit(X_train, y_train)
    
    print("\nTrên tập huấn luyện:")
    y_train_pred = model.predict(X_train)
    compute_metrics_updated(y_train, y_train_pred, f"{model_name} (Train)")
    
    print("\nTrên tập kiểm tra:")
    y_test_pred = model.predict(X_test)
    acc_test, f1_test = compute_metrics_updated(y_test, y_test_pred, f"{model_name} (Test)")

    # Lưu model nếu nó tốt nhất
    if f1_test > best_f1_score_global:
        best_f1_score_global = f1_test
        best_model_path_global = os.path.join(MODEL_SAVE_DIR, f"best_sklearn_model_{model_name.replace(' ', '_')}.joblib")
        best_model_type_global = 'sklearn'
        try:
            joblib.dump(model, best_model_path_global)
            print(f"Đã lưu model ML tốt nhất mới: {model_name} với F1 (Test): {f1_test:.4f} tại {best_model_path_global}")
        except Exception as e:
            print(f"Lỗi khi lưu model {model_name}: {e}")
            
    return acc_test, f1_test, model

In [ ]:
MAX_FEATURES_ML = 2500

# 1. CountVectorizer
print("Vectorizing với CountVectorizer...")
count_vec_stm = CountVectorizer(max_features=MAX_FEATURES_ML)
X_train_stm_count = count_vec_stm.fit_transform(train_stm_text)
X_test_stm_count = count_vec_stm.transform(test_stm_text)

count_vec_lem = CountVectorizer(max_features=MAX_FEATURES_ML)
X_train_lem_count = count_vec_lem.fit_transform(train_lem_text)
X_test_lem_count = count_vec_lem.transform(test_lem_text)

# 2. TfidfVectorizer
print("Vectorizing với TfidfVectorizer...")
tfidf_vec_stm = TfidfVectorizer(max_features=MAX_FEATURES_ML)
X_train_stm_tfidf = tfidf_vec_stm.fit_transform(train_stm_text)
X_test_stm_tfidf = tfidf_vec_stm.transform(test_stm_text)

tfidf_vec_lem = TfidfVectorizer(max_features=MAX_FEATURES_ML)
X_train_lem_tfidf = tfidf_vec_lem.fit_transform(train_lem_text)
X_test_lem_tfidf = tfidf_vec_lem.transform(test_lem_text)

# 3. Word2Vec
print("Huấn luyện Word2Vec và tạo document vectors...")
W2V_SIZE = 300
W2V_WINDOW = 5
W2V_MIN_COUNT = 3
W2V_WORKERS = 4

def create_document_vector(tokens_list_series, model, num_features):
    document_vectors = np.zeros((len(tokens_list_series), num_features))
    for i, tokens in enumerate(tokens_list_series): # tokens_list_series là Series of lists
        feature_vec = np.zeros((num_features,), dtype="float32")
        nwords = 0
        for word in tokens: # tokens ở đây là list các từ của 1 document
            if word in model.wv:
                nwords = nwords + 1
                feature_vec = np.add(feature_vec, model.wv[word])
        if nwords > 0:
            feature_vec = np.divide(feature_vec, nwords)
        document_vectors[i] = feature_vec
    return document_vectors

# Stemming
w2v_model_stm = Word2Vec(sentences=train_stm_tokens, vector_size=W2V_SIZE, window=W2V_WINDOW, min_count=W2V_MIN_COUNT, workers=W2V_WORKERS)
X_train_stm_w2v = create_document_vector(train_stm_tokens, w2v_model_stm, W2V_SIZE)
X_test_stm_w2v = create_document_vector(test_stm_tokens, w2v_model_stm, W2V_SIZE)

# Lemmatization
w2v_model_lem = Word2Vec(sentences=train_lem_tokens, vector_size=W2V_SIZE, window=W2V_WINDOW, min_count=W2V_MIN_COUNT, workers=W2V_WORKERS)
X_train_lem_w2v = create_document_vector(train_lem_tokens, w2v_model_lem, W2V_SIZE)
X_test_lem_w2v = create_document_vector(test_lem_tokens, w2v_model_lem, W2V_SIZE)

# 4. FastText
print("Huấn luyện FastText và tạo document vectors...")
# Stemming
ft_model_stm = FastText(sentences=train_stm_tokens, vector_size=W2V_SIZE, window=W2V_WINDOW, min_count=W2V_MIN_COUNT, workers=W2V_WORKERS, sg=1)
X_train_stm_ft = create_document_vector(train_stm_tokens, ft_model_stm, W2V_SIZE)
X_test_stm_ft = create_document_vector(test_stm_tokens, ft_model_stm, W2V_SIZE)

# Lemmatization
ft_model_lem = FastText(sentences=train_lem_tokens, vector_size=W2V_SIZE, window=W2V_WINDOW, min_count=W2V_MIN_COUNT, workers=W2V_WORKERS, sg=1)
X_train_lem_ft = create_document_vector(train_lem_tokens, ft_model_lem, W2V_SIZE)
X_test_lem_ft = create_document_vector(test_lem_tokens, ft_model_lem, W2V_SIZE)

print("Đã hoàn thành Vectorization cho ML.")

In [ ]:
# Cell 5: Huấn luyện các mô hình ML truyền thống

results_ml = [] 

ml_configurations = [
    # CountVectorizer
    {"name": "LR_CountVec_Stm", "model": LogisticRegression(solver='liblinear', class_weight='balanced', C=1.0, max_iter=1000), "X_train": X_train_stm_count, "X_test": X_test_stm_count, "y_train": y_train_stm, "y_test": y_test_stm},
    {"name": "LR_CountVec_Lem", "model": LogisticRegression(solver='liblinear', class_weight='balanced', C=1.0, max_iter=1000), "X_train": X_train_lem_count, "X_test": X_test_lem_count, "y_train": y_train_lem, "y_test": y_test_lem},
    {"name": "MNB_CountVec_Stm", "model": MultinomialNB(alpha=1.0), "X_train": X_train_stm_count, "X_test": X_test_stm_count, "y_train": y_train_stm, "y_test": y_test_stm},
    {"name": "MNB_CountVec_Lem", "model": MultinomialNB(alpha=1.0), "X_train": X_train_lem_count, "X_test": X_test_lem_count, "y_train": y_train_lem, "y_test": y_test_lem},
    # SỬA DUAL Ở ĐÂY
    {"name": "LSVC_CountVec_Stm", "model": LinearSVC(class_weight='balanced', dual=False, C=1.0, max_iter=2000), "X_train": X_train_stm_count, "X_test": X_test_stm_count, "y_train": y_train_stm, "y_test": y_test_stm}, 
    {"name": "LSVC_CountVec_Lem", "model": LinearSVC(class_weight='balanced', dual=False, C=1.0, max_iter=2000), "X_train": X_train_lem_count, "X_test": X_test_lem_count, "y_train": y_train_lem, "y_test": y_test_lem},
    {"name": "KNN_CountVec_Stm", "model": KNeighborsClassifier(n_neighbors=10), "X_train": X_train_stm_count, "X_test": X_test_stm_count, "y_train": y_train_stm, "y_test": y_test_stm},
    {"name": "KNN_CountVec_Lem", "model": KNeighborsClassifier(n_neighbors=10), "X_train": X_train_lem_count, "X_test": X_test_lem_count, "y_train": y_train_lem, "y_test": y_test_lem},

    # TfidfVectorizer
    {"name": "LR_Tfidf_Stm", "model": LogisticRegression(solver='liblinear', class_weight='balanced', C=1.0, max_iter=1000), "X_train": X_train_stm_tfidf, "X_test": X_test_stm_tfidf, "y_train": y_train_stm, "y_test": y_test_stm},
    {"name": "LR_Tfidf_Lem", "model": LogisticRegression(solver='liblinear', class_weight='balanced', C=1.0, max_iter=1000), "X_train": X_train_lem_tfidf, "X_test": X_test_lem_tfidf, "y_train": y_train_lem, "y_test": y_test_lem},
    {"name": "MNB_Tfidf_Stm", "model": MultinomialNB(alpha=1.0), "X_train": X_train_stm_tfidf, "X_test": X_test_stm_tfidf, "y_train": y_train_stm, "y_test": y_test_stm},
    {"name": "MNB_Tfidf_Lem", "model": MultinomialNB(alpha=1.0), "X_train": X_train_lem_tfidf, "X_test": X_test_lem_tfidf, "y_train": y_train_lem, "y_test": y_test_lem},
    # SỬA DUAL Ở ĐÂY
    {"name": "LSVC_Tfidf_Stm", "model": LinearSVC(class_weight='balanced', dual=False, C=1.0, max_iter=2000), "X_train": X_train_stm_tfidf, "X_test": X_test_stm_tfidf, "y_train": y_train_stm, "y_test": y_test_stm},
    {"name": "LSVC_Tfidf_Lem", "model": LinearSVC(class_weight='balanced', dual=False, C=1.0, max_iter=2000), "X_train": X_train_lem_tfidf, "X_test": X_test_lem_tfidf, "y_train": y_train_lem, "y_test": y_test_lem},
    {"name": "KNN_Tfidf_Stm", "model": KNeighborsClassifier(n_neighbors=10), "X_train": X_train_stm_tfidf, "X_test": X_test_stm_tfidf, "y_train": y_train_stm, "y_test": y_test_stm},
    {"name": "KNN_Tfidf_Lem", "model": KNeighborsClassifier(n_neighbors=10), "X_train": X_train_lem_tfidf, "X_test": X_test_lem_tfidf, "y_train": y_train_lem, "y_test": y_test_lem},
    
    # Word2Vec (sử dụng ma trận dày)
    {"name": "LR_W2V_Stm", "model": LogisticRegression(solver='liblinear', class_weight='balanced', C=1.0, max_iter=1000), "X_train": X_train_stm_w2v, "X_test": X_test_stm_w2v, "y_train": y_train_stm, "y_test": y_test_stm},
    {"name": "LR_W2V_Lem", "model": LogisticRegression(solver='liblinear', class_weight='balanced', C=1.0, max_iter=1000), "X_train": X_train_lem_w2v, "X_test": X_test_lem_w2v, "y_train": y_train_lem, "y_test": y_test_lem},
    # SỬA DUAL Ở ĐÂY
    {"name": "LSVC_W2V_Stm", "model": LinearSVC(class_weight='balanced', dual=False, C=1.0, max_iter=2000), "X_train": X_train_stm_w2v, "X_test": X_test_stm_w2v, "y_train": y_train_stm, "y_test": y_test_stm},
    # Với Word2Vec, n_features (W2V_SIZE) có thể nhỏ, dual=True có thể phù hợp hơn.
    # Tuy nhiên, để nhất quán và tránh lỗi, ta thử dual=False trước.
    {"name": "LSVC_W2V_Lem", "model": LinearSVC(class_weight='balanced', dual=False, C=1.0, max_iter=2000), "X_train": X_train_lem_w2v, "X_test": X_test_lem_w2v, "y_train": y_train_lem, "y_test": y_test_lem},
    {"name": "KNN_W2V_Stm", "model": KNeighborsClassifier(n_neighbors=10), "X_train": X_train_stm_w2v, "X_test": X_test_stm_w2v, "y_train": y_train_stm, "y_test": y_test_stm},
    {"name": "KNN_W2V_Lem", "model": KNeighborsClassifier(n_neighbors=10), "X_train": X_train_lem_w2v, "X_test": X_test_lem_w2v, "y_train": y_train_lem, "y_test": y_test_lem},

    # FastText (sử dụng ma trận dày)
    {"name": "LR_FT_Stm", "model": LogisticRegression(solver='liblinear', class_weight='balanced', C=1.0, max_iter=1000), "X_train": X_train_stm_ft, "X_test": X_test_stm_ft, "y_train": y_train_stm, "y_test": y_test_stm},
    {"name": "LR_FT_Lem", "model": LogisticRegression(solver='liblinear', class_weight='balanced', C=1.0, max_iter=1000), "X_train": X_train_lem_ft, "X_test": X_test_lem_ft, "y_train": y_train_lem, "y_test": y_test_lem},
    # SỬA DUAL Ở ĐÂY
    {"name": "LSVC_FT_Stm", "model": LinearSVC(class_weight='balanced', dual=False, C=1.0, max_iter=2000), "X_train": X_train_stm_ft, "X_test": X_test_stm_ft, "y_train": y_train_stm, "y_test": y_test_stm},
    {"name": "LSVC_FT_Lem", "model": LinearSVC(class_weight='balanced', dual=False, C=1.0, max_iter=2000), "X_train": X_train_lem_ft, "X_test": X_test_lem_ft, "y_train": y_train_lem, "y_test": y_test_lem},
    {"name": "KNN_FT_Stm", "model": KNeighborsClassifier(n_neighbors=10), "X_train": X_train_stm_ft, "X_test": X_test_stm_ft, "y_train": y_train_stm, "y_test": y_test_stm},
    {"name": "KNN_FT_Lem", "model": KNeighborsClassifier(n_neighbors=10), "X_train": X_train_lem_ft, "X_test": X_test_lem_ft, "y_train": y_train_lem, "y_test": y_test_lem},
]

for config in ml_configurations:
    X_train_to_use = config["X_train"]
    X_test_to_use = config["X_test"]
    
    if "MNB" in config["name"] and ("W2V" in config["name"] or "FT" in config["name"]):
        print(f"Bỏ qua {config['name']} vì MNB không phù hợp với dense embeddings W2V/FT.")
        continue
        
    acc, f1, _ = train_and_evaluate_ml_model(
        config["model"],
        X_train_to_use,
        config["y_train"],
        X_test_to_use,
        config["y_test"],
        config["name"]
    )
    results_ml.append({"Model": config["name"], "Accuracy_Test": acc, "F1_Macro_Test": f1})

results_ml_df = pd.DataFrame(results_ml)
print("\n--- Bảng tổng kết kết quả ML truyền thống ---")
if not results_ml_df.empty:
    print(results_ml_df.sort_values(by="F1_Macro_Test", ascending=False))
else:
    print("Không có kết quả ML nào để hiển thị (results_ml_df rỗng).")

In [ ]:
# --- Phần Mô hình CNN ---
VOCAB_SIZE_CNN_BASE = 5000 # Kích thước vocab cơ sở cho CNN, sẽ được cập nhật theo vectorizer
EMBEDDING_DIM_CNN = 128
HIDDEN_SIZE_CNN = 64
MAX_SEQ_LENGTH_CNN = 200
DROPOUT_RATE_CNN = 0.4 # Tăng dropout
EPOCHS_CNN = 15 # Tăng epochs một chút
BATCH_SIZE_CNN = 64
LEARNING_RATE_CNN = 0.001

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nSử dụng thiết bị: {device} cho CNN")

class CNNClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_classes, dropout_rate, padding_idx=0):
        super(CNNClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        
        # Sử dụng nhiều kernel_size khác nhau
        self.convs = nn.ModuleList([
            nn.Conv1d(embedding_dim, hidden_size, kernel_size=ks, padding=(ks-1)//2) for ks in [3, 4, 5]
        ])
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        # Số lượng filters là hidden_size * số lượng conv layers
        self.fc = nn.Linear(len(self.convs) * hidden_size, num_classes)

    def forward(self, x): # x: (batch_size, seq_len)
        embedded = self.embedding(x)  # -> (batch_size, seq_len, embedding_dim)
        embedded = embedded.permute(0, 2, 1)  # -> (batch_size, embedding_dim, seq_len)
        
        # Áp dụng từng lớp conv và max pooling
        conved_outputs = []
        for conv in self.convs:
            conved = self.relu(conv(embedded)) # -> (batch_size, hidden_size, seq_len_after_conv)
            # Global Max Pooling over time
            pooled = torch.max(conved, dim=2)[0] # -> (batch_size, hidden_size)
            conved_outputs.append(pooled)
            
        # Nối output từ các filters khác nhau
        concatenated = torch.cat(conved_outputs, dim=1) # -> (batch_size, len(convs) * hidden_size)
        
        dropped = self.dropout(concatenated)
        output = self.fc(dropped) # -> (batch_size, num_classes)
        return output

def text_to_ids_cnn(text_tokens, vocab_map, max_seq_len, unk_id=0, pad_id=0):
    ids = [vocab_map.get(token, unk_id) for token in text_tokens]
    if len(ids) > max_seq_len:
        ids = ids[:max_seq_len]
    else:
        ids.extend([pad_id] * (max_seq_len - len(ids)))
    return ids

class TextDataset(Dataset):
    def __init__(self, tokens_series, labels_series, vocab_map, max_seq_len, unk_id=0, pad_id=0):
        self.labels = labels_series.tolist()
        self.texts_ids = [text_to_ids_cnn(tokens, vocab_map, max_seq_len, unk_id, pad_id) for tokens in tokens_series]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return torch.tensor(self.texts_ids[idx], dtype=torch.long), torch.tensor(self.labels[idx], dtype=torch.long)

def train_cnn_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss, total_acc, total_count = 0, 0, 0
    for text_batch, label_batch in dataloader:
        text_batch, label_batch = text_batch.to(device), label_batch.to(device)
        optimizer.zero_grad()
        predictions = model(text_batch)
        loss = criterion(predictions, label_batch)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        total_loss += loss.item()
        total_acc += (predictions.argmax(1) == label_batch).sum().item()
        total_count += label_batch.size(0)
    return total_loss / len(dataloader), total_acc / total_count

def evaluate_cnn(model, dataloader, criterion, device):
    model.eval()
    total_loss, total_acc, total_count = 0, 0, 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for text_batch, label_batch in dataloader:
            text_batch, label_batch = text_batch.to(device), label_batch.to(device)
            predictions = model(text_batch)
            loss = criterion(predictions, label_batch)
            total_loss += loss.item()
            total_acc += (predictions.argmax(1) == label_batch).sum().item()
            total_count += label_batch.size(0)
            all_preds.extend(predictions.argmax(1).cpu().tolist())
            all_labels.extend(label_batch.cpu().tolist())
    return total_loss / len(dataloader), total_acc / total_count, all_labels, all_preds

results_cnn = [] # List để lưu kết quả CNN# --- Phần Mô hình CNN ---
VOCAB_SIZE_CNN_BASE = 5000 # Kích thước vocab cơ sở cho CNN, sẽ được cập nhật theo vectorizer
EMBEDDING_DIM_CNN = 128
HIDDEN_SIZE_CNN = 64
MAX_SEQ_LENGTH_CNN = 200
DROPOUT_RATE_CNN = 0.4 # Tăng dropout
EPOCHS_CNN = 15 # Tăng epochs một chút
BATCH_SIZE_CNN = 64
LEARNING_RATE_CNN = 0.001

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nSử dụng thiết bị: {device} cho CNN")

class CNNClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_classes, dropout_rate, padding_idx=0):
        super(CNNClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        
        # Sử dụng nhiều kernel_size khác nhau
        self.convs = nn.ModuleList([
            nn.Conv1d(embedding_dim, hidden_size, kernel_size=ks, padding=(ks-1)//2) for ks in [3, 4, 5]
        ])
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        # Số lượng filters là hidden_size * số lượng conv layers
        self.fc = nn.Linear(len(self.convs) * hidden_size, num_classes)

    def forward(self, x): # x: (batch_size, seq_len)
        embedded = self.embedding(x)  # -> (batch_size, seq_len, embedding_dim)
        embedded = embedded.permute(0, 2, 1)  # -> (batch_size, embedding_dim, seq_len)
        
        # Áp dụng từng lớp conv và max pooling
        conved_outputs = []
        for conv in self.convs:
            conved = self.relu(conv(embedded)) # -> (batch_size, hidden_size, seq_len_after_conv)
            # Global Max Pooling over time
            pooled = torch.max(conved, dim=2)[0] # -> (batch_size, hidden_size)
            conved_outputs.append(pooled)
            
        # Nối output từ các filters khác nhau
        concatenated = torch.cat(conved_outputs, dim=1) # -> (batch_size, len(convs) * hidden_size)
        
        dropped = self.dropout(concatenated)
        output = self.fc(dropped) # -> (batch_size, num_classes)
        return output

def text_to_ids_cnn(text_tokens, vocab_map, max_seq_len, unk_id=0, pad_id=0):
    ids = [vocab_map.get(token, unk_id) for token in text_tokens]
    if len(ids) > max_seq_len:
        ids = ids[:max_seq_len]
    else:
        ids.extend([pad_id] * (max_seq_len - len(ids)))
    return ids

class TextDataset(Dataset):
    def __init__(self, tokens_series, labels_series, vocab_map, max_seq_len, unk_id=0, pad_id=0):
        self.labels = labels_series.tolist()
        self.texts_ids = [text_to_ids_cnn(tokens, vocab_map, max_seq_len, unk_id, pad_id) for tokens in tokens_series]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return torch.tensor(self.texts_ids[idx], dtype=torch.long), torch.tensor(self.labels[idx], dtype=torch.long)

def train_cnn_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss, total_acc, total_count = 0, 0, 0
    for text_batch, label_batch in dataloader:
        text_batch, label_batch = text_batch.to(device), label_batch.to(device)
        optimizer.zero_grad()
        predictions = model(text_batch)
        loss = criterion(predictions, label_batch)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        total_loss += loss.item()
        total_acc += (predictions.argmax(1) == label_batch).sum().item()
        total_count += label_batch.size(0)
    return total_loss / len(dataloader), total_acc / total_count

def evaluate_cnn(model, dataloader, criterion, device):
    model.eval()
    total_loss, total_acc, total_count = 0, 0, 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for text_batch, label_batch in dataloader:
            text_batch, label_batch = text_batch.to(device), label_batch.to(device)
            predictions = model(text_batch)
            loss = criterion(predictions, label_batch)
            total_loss += loss.item()
            total_acc += (predictions.argmax(1) == label_batch).sum().item()
            total_count += label_batch.size(0)
            all_preds.extend(predictions.argmax(1).cpu().tolist())
            all_labels.extend(label_batch.cpu().tolist())
    return total_loss / len(dataloader), total_acc / total_count, all_labels, all_preds

results_cnn = [] # List để lưu kết quả CNN

In [ ]:
def run_cnn_experiment(
    model_class, 
    train_tokens_series, test_tokens_series, 
    y_train_series, y_test_series,
    vocab_source_for_cnn, 
    preprocessing_type, 
    embedding_dim, hidden_size, num_classes_cnn, dropout_rate,
    max_seq_len, epochs, batch_size, lr,
    count_vectorizer_instance=None, 
    tfidf_vectorizer_instance=None,
    w2v_model_instance=None,
    ft_model_instance=None
    ):
    global best_f1_score_global, best_model_path_global, best_model_type_global # Khai báo sử dụng biến global
    
    model_full_name = f"CNN_{vocab_source_for_cnn.upper()}_{preprocessing_type.upper()}"
    print(f"\n--- Chạy thử nghiệm CNN: {model_full_name} ---")

    UNK_ID = 0 
    PAD_ID = 0 

    original_vectors = None # Khởi tạo để tránh lỗi nếu không phải w2v/ft

    if vocab_source_for_cnn == 'countvec':
        vocab_map = count_vectorizer_instance.vocabulary_
        actual_vocab_size = max(vocab_map.values()) + 1 if vocab_map else 1
    elif vocab_source_for_cnn == 'tfidfvec':
        vocab_map = tfidf_vectorizer_instance.vocabulary_
        actual_vocab_size = max(vocab_map.values()) + 1 if vocab_map else 1
    elif vocab_source_for_cnn == 'w2v':
        vocab_map_orig = w2v_model_instance.wv.key_to_index
        actual_vocab_size = len(w2v_model_instance.wv)
        idx = 0; standard_vocab_map = {}; original_vectors = []
        for word in w2v_model_instance.wv.index_to_key:
            standard_vocab_map[word] = idx
            original_vectors.append(w2v_model_instance.wv[word])
            idx +=1
        vocab_map = standard_vocab_map
        actual_vocab_size = len(vocab_map)
    elif vocab_source_for_cnn == 'ft':
        vocab_map_orig = ft_model_instance.wv.key_to_index
        actual_vocab_size = len(ft_model_instance.wv)
        idx = 0; standard_vocab_map = {}; original_vectors = []
        for word in ft_model_instance.wv.index_to_key:
            standard_vocab_map[word] = idx
            original_vectors.append(ft_model_instance.wv[word])
            idx +=1
        vocab_map = standard_vocab_map
        actual_vocab_size = len(vocab_map)
    else:
        raise ValueError("vocab_source_for_cnn không hợp lệ.")
    
    print(f"Kích thước vocab thực tế cho Embedding: {actual_vocab_size}")

    train_dataset = TextDataset(train_tokens_series, y_train_series, vocab_map, max_seq_len, UNK_ID, PAD_ID)
    test_dataset = TextDataset(test_tokens_series, y_test_series, vocab_map, max_seq_len, UNK_ID, PAD_ID)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    cnn_model = model_class(actual_vocab_size, embedding_dim, hidden_size, num_classes_cnn, dropout_rate, padding_idx=PAD_ID).to(device)
    
    if (vocab_source_for_cnn == 'w2v' or vocab_source_for_cnn == 'ft') and original_vectors is not None:
        pretrained_weights_np = np.array(original_vectors)
        # Đảm bảo rằng ma trận trọng số không rỗng và có đúng số chiều embedding
        if pretrained_weights_np.size > 0 and pretrained_weights_np.shape[1] == cnn_model.embedding.embedding_dim:
            # Nếu vocab size của model lớn hơn vocab của pretrain (ví dụ do thêm PAD/UNK token vào embedding layer)
            # Tạo ma trận mới và copy trọng số vào
            final_weights = torch.FloatTensor(cnn_model.embedding.num_embeddings, cnn_model.embedding.embedding_dim).uniform_(-0.05, 0.05)
            
            # Số lượng từ trong vocab đã chuẩn hóa của chúng ta (từ W2V/FT)
            num_pretrained_words = pretrained_weights_np.shape[0] 
            
            # Lớp embedding của chúng ta có thể lớn hơn (actual_vocab_size) nếu có token PAD/UNK riêng
            # Tuy nhiên, với logic hiện tại, actual_vocab_size = num_pretrained_words
            # Chúng ta sẽ copy trực tiếp nếu kích thước khớp
            if num_pretrained_words == actual_vocab_size:
                 final_weights = torch.FloatTensor(pretrained_weights_np)
            elif num_pretrained_words < actual_vocab_size:
                # Trường hợp này ít xảy ra với logic hiện tại nhưng để phòng ngừa
                final_weights[:num_pretrained_words, :] = torch.FloatTensor(pretrained_weights_np)
                print(f"Cảnh báo: Vocab của Embedding Layer ({actual_vocab_size}) lớn hơn vocab pretrain ({num_pretrained_words}). Phần còn lại được khởi tạo ngẫu nhiên.")
            else: # num_pretrained_words > actual_vocab_size, cắt bớt
                final_weights = torch.FloatTensor(pretrained_weights_np[:actual_vocab_size, :])
                print(f"Cảnh báo: Vocab pretrain ({num_pretrained_words}) lớn hơn vocab của Embedding Layer ({actual_vocab_size}). Đã cắt bớt pretrain.")


            if final_weights.shape[0] == cnn_model.embedding.num_embeddings:
                cnn_model.embedding.weight.data.copy_(final_weights)
                cnn_model.embedding.weight.requires_grad = True # Cho phép fine-tuning
                print(f"Đã khởi tạo trọng số Embedding từ {vocab_source_for_cnn.upper()}. Embedding trainable.")
            else:
                 print(f"LỖI kích thước: final_weights.shape[0] ({final_weights.shape[0]}) != cnn_model.embedding.num_embeddings ({cnn_model.embedding.num_embeddings}). Không khởi tạo.")

        elif original_vectors is not None: # original_vectors có nhưng không khớp điều kiện
            print(f"Cảnh báo: Không thể khởi tạo trọng số Embedding cho {model_full_name} từ pretrain. Kích thước không khớp hoặc pretrain rỗng.")
            print(f"  Pretrain shape: {np.array(original_vectors).shape}, Model embedding_dim: {cnn_model.embedding.embedding_dim}")


    optimizer = optim.Adam(cnn_model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    train_losses_epoch, train_accs_epoch = [], []
    test_losses_epoch, test_accs_epoch = [], []
    
    current_best_f1_epoch = -1.0
    best_epoch_model_state = None

    for epoch in range(epochs):
        train_loss, train_acc = train_cnn_epoch(cnn_model, train_dataloader, optimizer, criterion, device)
        # Đánh giá trên tập test sau mỗi epoch để có thể chọn model tốt nhất theo epoch
        test_loss, test_acc, epoch_y_true, epoch_y_pred = evaluate_cnn(cnn_model, test_dataloader, criterion, device)
        epoch_f1 = f1_score(epoch_y_true, epoch_y_pred, average='macro') # Tính F1 cho epoch này
        
        train_losses_epoch.append(train_loss)
        train_accs_epoch.append(train_acc)
        test_losses_epoch.append(test_loss)
        test_accs_epoch.append(test_acc)
        
        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}% | Test Loss: {test_loss:.4f}, Test Acc: {test_acc*100:.2f}%, Test F1: {epoch_f1:.4f}")

        # Lưu state của model tốt nhất trong các epoch của lần chạy CNN này
        if epoch_f1 > current_best_f1_epoch:
            current_best_f1_epoch = epoch_f1
            best_epoch_model_state = cnn_model.state_dict()
            print(f"*** New best F1 within this CNN run: {current_best_f1_epoch:.4f} at epoch {epoch+1} ***")


    # Sau khi hoàn thành tất cả các epoch, load lại state tốt nhất và đánh giá lần cuối
    if best_epoch_model_state:
        cnn_model.load_state_dict(best_epoch_model_state)
        print(f"Đã tải lại model từ epoch có F1 tốt nhất ({current_best_f1_epoch:.4f}) để đánh giá cuối cùng.")
    
    _, _, final_y_true, final_y_pred = evaluate_cnn(cnn_model, test_dataloader, criterion, device)
    acc_final, f1_final = compute_metrics_updated(final_y_true, final_y_pred, model_full_name + " (Test Final)")
    
    # Lưu model PyTorch nếu nó tốt nhất toàn cục
    if f1_final > best_f1_score_global:
        best_f1_score_global = f1_final
        model_filename = f"best_pytorch_model_{model_full_name.replace(' ', '_')}.pth"
        best_model_path_global = os.path.join(MODEL_SAVE_DIR, model_filename)
        best_model_type_global = 'pytorch'
        
        # Lưu state_dict và các thông tin cần thiết để tải lại
        checkpoint = {
            'model_state_dict': cnn_model.state_dict(), # Hoặc best_epoch_model_state nếu bạn muốn lưu state đó
            'vocab_source': vocab_source_for_cnn,
            'preprocessing_type': preprocessing_type,
            'actual_vocab_size': actual_vocab_size, # Quan trọng
            'embedding_dim': embedding_dim,
            'hidden_size': hidden_size,
            'num_classes': num_classes_cnn,
            'dropout_rate': dropout_rate,
            'padding_idx': PAD_ID,
            'model_architecture_name': model_class.__name__, # Tên lớp model
            'label_encoder_classes': list(le.classes_) if 'le' in globals() and hasattr(le, 'classes_') else None # Lưu các lớp của LabelEncoder
        }
        # Lưu vocab_map nếu là countvec hoặc tfidfvec để có thể tái tạo mapping
        if vocab_source_for_cnn in ['countvec', 'tfidfvec']:
            checkpoint['vocab_map'] = vocab_map
        # Đối với W2V/FT, việc lưu toàn bộ model Gensim có thể tốt hơn nếu muốn dùng lại,
        # nhưng ở đây ta chỉ lưu vocab_map đã chuẩn hóa cho PyTorch model.
        elif vocab_source_for_cnn in ['w2v', 'ft']:
             checkpoint['vocab_map'] = vocab_map # vocab_map đã được chuẩn hóa

        try:
            torch.save(checkpoint, best_model_path_global)
            print(f"Đã lưu model PyTorch tốt nhất mới: {model_full_name} với F1 (Test): {f1_final:.4f} tại {best_model_path_global}")
        except Exception as e:
            print(f"Lỗi khi lưu model PyTorch {model_full_name}: {e}")
            
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1); plt.plot(train_losses_epoch, label='Train Loss'); plt.plot(test_losses_epoch, label='Test Loss'); plt.title(f'Loss - {model_full_name}'); plt.legend()
    plt.subplot(1, 2, 2); plt.plot(train_accs_epoch, label='Train Acc'); plt.plot(test_accs_epoch, label='Test Acc'); plt.title(f'Accuracy - {model_full_name}'); plt.legend()
    plt.tight_layout(); plt.show()
    
    return acc_final, f1_final, cnn_model

In [ ]:
results_cnn = []

cnn_configurations_setup = [
    {"vocab_source": "countvec", "preprocessing_type": "stm", "train_tokens": train_stm_tokens, "test_tokens": test_stm_tokens, "y_train": y_train_stm, "y_test": y_test_stm, "vectorizer_instance": count_vec_stm},
    {"vocab_source": "countvec", "preprocessing_type": "lem", "train_tokens": train_lem_tokens, "test_tokens": test_lem_tokens, "y_train": y_train_lem, "y_test": y_test_lem, "vectorizer_instance": count_vec_lem},
    
    {"vocab_source": "tfidfvec", "preprocessing_type": "stm", "train_tokens": train_stm_tokens, "test_tokens": test_stm_tokens, "y_train": y_train_stm, "y_test": y_test_stm, "vectorizer_instance": tfidf_vec_stm},
    {"vocab_source": "tfidfvec", "preprocessing_type": "lem", "train_tokens": train_lem_tokens, "test_tokens": test_lem_tokens, "y_train": y_train_lem, "y_test": y_test_lem, "vectorizer_instance": tfidf_vec_lem},

    {"vocab_source": "w2v", "preprocessing_type": "stm", "train_tokens": train_stm_tokens, "test_tokens": test_stm_tokens, "y_train": y_train_stm, "y_test": y_test_stm, "w2v_model_instance": w2v_model_stm},
    {"vocab_source": "w2v", "preprocessing_type": "lem", "train_tokens": train_lem_tokens, "test_tokens": test_lem_tokens, "y_train": y_train_lem, "y_test": y_test_lem, "w2v_model_instance": w2v_model_lem},

    {"vocab_source": "ft", "preprocessing_type": "stm", "train_tokens": train_stm_tokens, "test_tokens": test_stm_tokens, "y_train": y_train_stm, "y_test": y_test_stm, "ft_model_instance": ft_model_stm},
    {"vocab_source": "ft", "preprocessing_type": "lem", "train_tokens": train_lem_tokens, "test_tokens": test_lem_tokens, "y_train": y_train_lem, "y_test": y_test_lem, "ft_model_instance": ft_model_lem},
]

for config in cnn_configurations_setup:
    acc, f1, _ = run_cnn_experiment(
        model_class=CNNClassifier,
        train_tokens_series=config["train_tokens"],
        test_tokens_series=config["test_tokens"],
        y_train_series=config["y_train"],
        y_test_series=config["y_test"],
        vocab_source_for_cnn=config["vocab_source"],
        preprocessing_type=config["preprocessing_type"],
        embedding_dim=EMBEDDING_DIM_CNN,
        hidden_size=HIDDEN_SIZE_CNN,
        num_classes_cnn=num_classes,
        dropout_rate=DROPOUT_RATE_CNN,
        max_seq_len=MAX_SEQ_LENGTH_CNN,
        epochs=EPOCHS_CNN,
        batch_size=BATCH_SIZE_CNN,
        lr=LEARNING_RATE_CNN,
        count_vectorizer_instance=config.get("vectorizer_instance") if config["vocab_source"] == "countvec" else None,
        tfidf_vectorizer_instance=config.get("vectorizer_instance") if config["vocab_source"] == "tfidfvec" else None,
        w2v_model_instance=config.get("w2v_model_instance"),
        ft_model_instance=config.get("ft_model_instance")
    )
    model_name_res = f"CNN_{config['vocab_source'].upper()}_{config['preprocessing_type'].upper()}"
    results_cnn.append({"Model": model_name_res, "Accuracy_Test": acc, "F1_Macro_Test": f1})

results_cnn_df = pd.DataFrame(results_cnn)
print("\n--- Bảng tổng kết kết quả CNN ---")
print(results_cnn_df.sort_values(by="F1_Macro_Test", ascending=False))

In [ ]:
if not results_ml_df.empty and not results_cnn_df.empty:
    all_results_df = pd.concat([results_ml_df, results_cnn_df], ignore_index=True)
elif not results_ml_df.empty:
    all_results_df = results_ml_df.copy()
elif not results_cnn_df.empty:
    all_results_df = results_cnn_df.copy()
else:
    all_results_df = pd.DataFrame()

if not all_results_df.empty:
    print("\n--- Bảng tổng kết TẤT CẢ KẾT QUẢ ---")
    print(all_results_df.sort_values(by="F1_Macro_Test", ascending=False))
    
    print(f"\n--- Model tốt nhất toàn cục ---")
    print(f"Loại model: {best_model_type_global}")
    print(f"F1 Score (Macro) trên tập Test: {best_f1_score_global:.4f}")
    print(f"Đường dẫn lưu model: {best_model_path_global}")
    if best_model_type_global == 'pytorch':
        print("Để tải lại model PyTorch, bạn cần định nghĩa lại kiến trúc model và sử dụng torch.load() trên checkpoint.")
    elif best_model_type_global == 'sklearn':
        print("Để tải lại model scikit-learn, sử dụng: loaded_model = joblib.load(best_model_path_global)")

else:
    print("Không có kết quả nào để hiển thị.")